In [1]:
import numpy as np
import matplotlib
import pandas as pd
import tensorflow as tf

dataset = pd.read_csv('/Users/wzhang/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data (1).csv')

train = dataset.iloc[:743,:]
test = dataset.iloc[743:,:]
print(train)
print(test)
X_train = pd.DataFrame(train.iloc[:,4:15].values)
Y_train = train.iloc[:,16:].values
print(X_train.shape)
print(Y_train.shape)
X_test = pd.DataFrame(test.iloc[:,4:15].values)
Y_test = test.iloc[:,16:].values
print(X_test.shape)
print(Y_test.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



2024-07-01 14:05:10.479166: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


          Date  Site  Latitude (decimal deg)  Longitude (decimal deg)  \
0    5/29/2013   WE6                 41.7134                 -83.3804   
1    5/29/2013   WE8                 41.8343                 -83.3633   
2    5/29/2013   WE4                 41.8267                 -83.1930   
3    6/10/2013   WE6                 41.7144                 -83.3805   
4    6/10/2013   WE8                 41.8353                 -83.3631   
..         ...   ...                     ...                      ...   
738  10/7/2019  WE16                 41.6599                 -83.1467   
739  10/7/2019  WE13                 41.7428                 -83.1354   
740  10/7/2019  WE12                 41.7023                 -83.2639   
741  10/7/2019   WE4                 41.8270                 -83.1949   
742  6/16/2020   WE6                 41.7054                 -83.3864   

     Secchi Depth (m)  CTD Temperature (°C)  \
0                1.60                  18.5   
1                1.00        

In [2]:
from keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, GRU, LayerNormalization, MultiHeadAttention,TimeDistributed, Input, Flatten
from sklearn.ensemble import RandomForestRegressor

In [3]:
def create_rf():
    # Initialize Random Forest regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, Y_train)
    return rf_model




In [4]:
from sklearn.svm import SVR
def create_SVM():
    # Initialize SVM regressor
    svm_model = SVR(kernel='rbf', C=10000, gamma='scale')

    # Train the model
    svm_model.fit(X_train, Y_train)
    return svm_model


In [5]:
rf = create_rf()
svm = create_SVM()

# ridge = create_ridge()
# svm = create_SVM()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
import deepforest
from deepforest import CascadeForestRegressor

In [7]:
from tensorflow.keras.layers import concatenate


# gruPredict = gru.predict(X_train)
# lassoPredict = lasso.predict(X_train)
# lstmPredict = lstm.predict(X_train)
rfPredict = rf.predict(X_train)
svmPredict = svm.predict(X_train)
# ridgePredict = ridge.predict(X_train)
# svmPredict = svm.predict(X_train)
# rfPredict = rf.predict(X_train)
# lassoPredict.resize(671,1)
rfPredict.resize(743,1)
svmPredict.resize(743,1)
# svmPredict.resize(671,1)
trainData = concatenate([rfPredict,svmPredict])

In [8]:
model = CascadeForestRegressor(n_estimators = 50, criterion='squared_error', backend='sklearn')

In [9]:
model.fit(trainData, Y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[2024-07-01 19:05:14.788] Start to fit the model:
[2024-07-01 19:05:14.788] Fitting cascade layer = 0 
[2024-07-01 19:05:34.781] layer = 0  | Val MSE = 162.58667 | Elapsed = 19.993 s
[2024-07-01 19:05:34.858] Fitting cascade layer = 1 
[2024-07-01 19:09:11.991] layer = 1  | Val MSE = 174.28390 | Elapsed = 217.134 s
[2024-07-01 19:09:11.991] Early stopping counter: 1 out of 2
[2024-07-01 19:09:12.071] Fitting cascade layer = 2 
[2024-07-01 19:12:45.614] layer = 2  | Val MSE = 198.26931 | Elapsed = 213.544 s
[2024-07-01 19:12:45.615] Early stopping counter: 2 out of 2
[2024-07-01 19:12:45.615] Handling early stopping
[2024-07-01 19:12:45.699] The optimal number of layers: 1


In [10]:
# gruPredictTest = gru.predict(X_test)
# lassoPredictTest = lasso.predict(X_test)
# lstmPredictTest = lstm.predict(X_test)
rfPredictTest = rf.predict(X_test)
svmPredictTest = svm.predict(X_test)
# ridgePredictTest = ridge.predict(X_test)
# svmPredictTest = svm.predict(X_test)
# svmPredictTest.resize(168,1)
# svmPredictTest.resize(96,1)
# lassoPredictTest.resize(168,1)
rfPredictTest.resize(96,1)
svmPredictTest.resize(96,1)
testData = concatenate([rfPredictTest, svmPredictTest])

In [11]:
res = model.predict(testData)
print(res)

[2024-07-01 19:12:45.987] Start to evalute the model:
[2024-07-01 19:12:45.988] Evaluating cascade layer = 0 
[[ 7.589066  ]
 [23.17398456]
 [ 4.149265  ]
 [ 5.08138   ]
 [ 8.044208  ]
 [29.769974  ]
 [ 8.0844477 ]
 [ 8.14984229]
 [11.800751  ]
 [32.67889701]
 [ 3.931905  ]
 [33.51409206]
 [11.559355  ]
 [31.619631  ]
 [ 4.87463654]
 [49.417523  ]
 [10.06360609]
 [47.343788  ]
 [ 9.23775269]
 [39.634136  ]
 [11.50708433]
 [54.969759  ]
 [ 5.38652   ]
 [12.55478697]
 [ 9.278308  ]
 [13.89777233]
 [61.669592  ]
 [22.466334  ]
 [58.663224  ]
 [ 9.602214  ]
 [16.449434  ]
 [39.241209  ]
 [ 2.581229  ]
 [41.610668  ]
 [22.003593  ]
 [43.071946  ]
 [14.66718478]
 [12.00653792]
 [37.9004006 ]
 [ 4.21046   ]
 [45.415339  ]
 [26.723628  ]
 [72.7128264 ]
 [25.90649823]
 [19.195653  ]
 [26.713118  ]
 [ 6.008768  ]
 [45.897296  ]
 [48.173154  ]
 [73.020036  ]
 [21.435933  ]
 [10.201799  ]
 [13.709709  ]
 [ 9.7236156 ]
 [25.162924  ]
 [33.27332949]
 [51.673442  ]
 [16.269895  ]
 [ 8.308171  ]
 [25.

In [12]:
from sklearn.metrics import r2_score 
r2 = r2_score(Y_test, res) 
print(r2)

0.8351806146547425


In [13]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test,res)
print(mae)

4.944228898813655


In [14]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(Y_test,res)
print(mape)

0.3897798394840167


In [15]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(Y_test,res)
print(rmse)

7.431193062418986
